# BUAD 313 - Spring 2025 - Assignment 4 (95 points)

Notes:
 - You may work in teams of up to 3.  Submit one assignment for the three of you, but please ensure it has all 3 of your names and @usc.edu emails.
 - You must submit your work as a .ipynb file (jupyter notebook). The grader has to be able to run your notebook. Code that doesn't run gets zero points.  A Great way to check that your code runs is to Select "Clear All Outputs", "Restart" and then "Run All" and make sure it still shows all your answer properly!
 - Use the existing sections below to submit your answers below.  You can add additional Python/markdown cells to describe and explain your solution, but keep it tidy.  Consider formatting some of your markdown cells for the grader.  [Markdown Guide](https://www.markdownguide.org/basic-syntax/)
 - For some of hte modeling quesitons, you may prefer to write your solution with paper and pencil and then include a photo in the markdown.  That's OK! Just please remember to include the file for your photo in your submission so that it renders proeprly.

The deadline for this assignment is **11:59 PM Pacific Time on Friday April 11, 2025**. Late submissions will not be accepted. (note the extended deadline!)

Below are the standard Python packages that we use for optimization models in this course. By running this next Python cell, you will have these packages available to use in all your answers contained in this file.

In [3]:
import numpy as np
from gurobipy import Model, GRB, quicksum
import pandas as pd

## Team Names and Emails:
 <font color="blue">**(Edit this cell)**</font>
 - William Jou: wcjou@usc.edu
 - Bain Higgins: cbhiggin@usc.edu
 - Jiya Valiram: jvaliram@usc.edu

## Question 1 (40 Points): E-Commerce Fulfillment (A Mixed Binary Perspective)

SwiftShip is a fast-growing e-commerce company that delivers products throughout the United States. As the company plans for the upcoming year, it must decide which of **five potential fulfillment centers** to rent and use for shipping products to its customers. Renting a fulfillment center comes with a **fixed annual cost**, which includes facility operations, staffing, and infrastructure.

Customer demand for products is **random and fluctuates daily**, but SwiftShip’s analytics team has provided **forecasted annual demand** for **20 customer zones** spread across the country. These forecasts represent the total number of units expected to be shipped to each zone in the upcoming year.

Each fulfillment center has a **maximum shipping capacity** — the total number of units it can process and send out over the year. Products can be shipped from any active fulfillment center to any customer zone. However, **shipping costs vary** depending on the distance between the fulfillment center and the customer zone.

SwiftShip’s goal is to meet the forecasted demand for all 20 zones **at the lowest possible total cost**, which includes both:

- **Rental costs** for any fulfillment centers they choose to activate
- **Shipping costs** based on how many units are sent from each center to each zone


### Potential Fulfillment Center Locations (5 cities):

- Atlanta, GA  
- Chicago, IL  
- Dallas, TX  
- Los Angeles, CA  
- Philadelphia, PA

### Customer Zones (20 regions):

- **Northeast**: Boston, MA; New York, NY; Washington, DC; Buffalo, NY  
- **Southeast**: Miami, FL; Charlotte, NC; Nashville, TN; Birmingham, AL  
- **Midwest**: Detroit, MI; Minneapolis, MN; St. Louis, MO; Cleveland, OH  
- **South Central**: Houston, TX; New Orleans, LA; Oklahoma City, OK  
- **West**: Phoenix, AZ; Denver, CO; Seattle, WA; San Francisco, CA; Salt Lake City, UT

I've provided some data that you might find useful as .csv files:
 - Rental Costs (in thousands of dolalrs per year) and Capacities (in thousands of units per year) for the 5 fulfillment centers
 - Forecasted Demand (in thousands of units) for each of the 20 customer zones
 - Shipping Costs (in dollars per unit) from each center to each zone

And because I'm a very generous professor, I've even done the data-wrangling to get these into dictionaries for you. 

In [4]:
import pandas as pd

# Read CSVs
rental_df = pd.read_csv("rental_costs_and_capacities.csv", index_col=0)
demand_df = pd.read_csv("customer_zone_demands.csv", index_col=0)
shipping_df = pd.read_csv("shipping_costs.csv", index_col=0)

# Helper to abbreviate city name (drop state abbreviation)
def abbreviate(name):
    return name.split(",")[0]

# 1. Rental costs dictionary
rental_costs = {abbreviate(city): cost for city, cost in rental_df["Rental_Cost_$K"].items()}

# 2. Capacity dictionary
capacity = {abbreviate(city): cap for city, cap in rental_df["Capacity_1000_Units"].items()}

# 3. Demand dictionary
demand = {abbreviate(zone): d for zone, d in demand_df["Demand_1000_Units"].items()}

# 4. Shipping costs dictionary (keys are (facility, zone))
shipping_costs = {
    (abbreviate(fac), abbreviate(zone)): shipping_df.loc[fac, zone]
    for fac in shipping_df.index
    for zone in shipping_df.columns
}

# Optional: check a few values
print("Rental Costs:", rental_costs)
print("Capacity:", capacity)
print("Demand:", demand)
print("Shipping Costs (sample):", dict(list(shipping_costs.items())[:5]))


Rental Costs: {'Atlanta': 502, 'Chicago': 670, 'Dallas': 506, 'Los Angeles': 471, 'Philadelphia': 588}
Capacity: {'Atlanta': 120, 'Chicago': 182, 'Dallas': 186, 'Los Angeles': 174, 'Philadelphia': 174}
Demand: {'Boston': 13, 'New York': 17, 'Washington': 12, 'Buffalo': 11, 'Miami': 21, 'Charlotte': 15, 'Nashville': 11, 'Birmingham': 10, 'Detroit': 21, 'Minneapolis': 21, 'St. Louis': 26, 'Cleveland': 19, 'Houston': 25, 'New Orleans': 24, 'Oklahoma City': 24, 'Phoenix': 28, 'Denver': 21, 'Seattle': 29, 'San Francisco': 12, 'Salt Lake City': 14}
Shipping Costs (sample): {('Atlanta', 'Boston'): 6.86, ('Atlanta', 'New York'): 3.36, ('Atlanta', 'Washington'): 2.52, ('Atlanta', 'Buffalo'): 9.59, ('Atlanta', 'Miami'): 9.73}


#### Part a (20 points)
Write a mathematical formulation for this problem.  If you want to write it by hand, be sure to write some markdown here that includes your photo as part of the notebook.  (See the markdown guide above for help.)

Your mathematical formulation should be clear and indicate what variables are binary, integer, and continuous.  Be clear about indexing and include any English descriptions/intuition you think make the formulation clearer for me and the Grader.  

## Part A

### Decision Variables
- Let F_i be a binary variable that is 1 when fufillment center i is opened and 0 when otherwise
- Let x_i_z be the number of units shipped from fufillment center i to customer zone j

### Objective Function
We want to minimize the cost 
- let R_i = the annual rental cost for fufillment center i
- let D_j = the forecasted demand for customer zone j
- let S_i_z = the shipping cost per unit from fufillment center i to customer zone j
- let C_i = the annual shipping capacity of fufillment center i 

Min((sum(R_i * F_i) for all in in fufillment centers) + (sum(R_i * S_i_z * x_i_z) for all i in fufillment centers and all z in customer zones)

### Constraints
- F_i must be 0 or 1 for all i
- Cannot ship negative units: x_i_z >= 0 for all i in fufillment centers and all z in customer zones
- The number of units shipped to each zone must atleast meet the demand for that zone: Sum(x_i_z for all i in fufillment centers) >= D_z for all z in customer zones
- The number of units shipped from a fufillment center must not exceed the shipment capacity of that fufillment zone: Sum(x_i_z for all z in customer zones) <= C_i * F_i for all i in fufillment centers

### Part b (20 points) 
Implement your model in Gurobi.  You should print out:
1. Which facilities you will rent.
2. The total rental costs. 
3. The total forecasted shipping costs.

In your implementation, you should ASSUME that the amount of product shipped from each facility can be fractional in order to simplify things a bit.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import Model, GRB, quicksum

model = Model("Facility Location Problem")

# DECISION VARIABLES
# create a binary variable for each facility (1 if facility is opened, 0 otherwise)
F = model.addVars(rental_costs.keys(), vtype=GRB.BINARY, name="Facility_Opened")
# create a decision variable x for the number of units shipped from each facility to each zone
x = model.addVars(shipping_costs.keys(), vtype=GRB.CONTINUOUS, lb=0, name="Units_Shipped")

# OBJECTIVE FUNCTION
# minimize total cost (rental + shipping)
model.setObjective(
    quicksum(rental_costs[i] * F[i] for i in rental_costs.keys()) +
    quicksum(shipping_costs[i, z] * x[i, z] for i, z in shipping_costs.keys()),
    GRB.MINIMIZE
)

# CONSTRAINTS
# ensure demand is met in each zone
for z in demand.keys():
    model.addConstr(
        quicksum(x[i, z] for i in rental_costs.keys()) >= demand[z],
        name=f"Demand_Constraint_{z}"
    )

# ensure capacity is not exceeded at each facility
for i in rental_costs.keys():
    model.addConstr(
        quicksum(x[i, z] for z in demand.keys()) <= capacity[i] * F[i],
        name=f"Capacity_Constraint_{i}"
    )

# ensure that if a facility is opened, it can ship to all zones
for i in rental_costs.keys():
    for z in demand.keys():
        model.addConstr(
            x[i, z] <= capacity[i] * F[i],
            name=f"Facility_Opened_Constraint_{i}_{z}"
        )


# optimize the model
model.optimize()

# print what facilities are opened
for i in rental_costs.keys():
    if F[i].x > 0.5:  # if the facility is opened
        print(f"Facility {i} is opened.")

# print the total rental costs
print(f"Total Rental Costs: {model.ObjVal - quicksum(shipping_costs[i, z] * x[i, z].x for i, z in shipping_costs.keys())}K")

#print the total forecasted shipping costs
print(f"Total Shipping Costs: {quicksum(shipping_costs[i, z] * x[i, z].x for i, z in shipping_costs.keys())}K")


Set parameter Username
Set parameter LicenseID to value 2614665
Academic license - for non-commercial use only - expires 2026-01-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 265K, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 125 rows, 105 columns and 405 nonzeros
Model fingerprint: 0x5be2ec81
Variable types: 100 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e+00, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 3e+01]
Presolve removed 100 rows and 0 columns
Presolve time: 0.00s
Presolved: 25 rows, 105 columns, 205 nonzeros
Variable types: 100 continuous, 5 integer (5 binary)
Found heuristic solution: objective 4085.0200000

Root relaxation: objective 2.523372e+03, 32 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current No

---
## Question 2 (55 points): Using Simulation to Better Assess a Recommendation.



In the previous question, you determined an optimal fulfillment plan under the assumption that forecasted **annual** demand is known with certainty. However, in practice, demand fluctuates from month to month, and total annual demand may deviate from the forecast.

SwiftShip has committed to renting fulfillment centers in **Atlanta**, **Dallas**, and **Los Angeles** for the upcoming year. In addition, SwiftShip has contracted with a third-party logistics (3PL) firm that can deliver units directly from a central factory to any customer zone **at a cost of \$20 per unit**. Since this option is expensive, SwiftShip prefers to use it **only when** the fulfillment centers are unable to satisfy monthly demand due to capacity constraints.

The company wants to better understand how much it might realistically spend on shipping over the year, given these sources of variability. To support this analysis, you will conduct a **simulation study** to estimate the *realized annual shipping cost* under random monthly demand.

### Assumptions:
- The **monthly capacity** of each fulfillment center is one-twelfth (1/12) of its annual capacity.
- Monthly demand in each customer zone is modeled as **normally distributed**, with:
  - Mean equal to **1/12 of the forecasted annual demand**, and  
  - Standard deviation equal to **40% of the monthly mean**.
  - Monthly demand values are **capped at zero from below** (i.e., if a simulated demand is negative, treat it as zero).
  - Monthly demand across zones is assumed independent. 
  
- Demand should be fulfilled in the **least costly way**, using:
  - The three active fulfillment centers (Atlanta, Dallas, Los Angeles), or  
  - The third-party firm (at \$20/unit) if necessary


#### Part a (20 points)

Remember that simulation modeling involves two key steps:

1. **Modeling uncertainty** — deciding how to represent uncertain quantities (like demand) as random variables.
2. **Outcome evaluation** — computing the outcomes of interest (like cost or service levels) as a function of those random variables and your decisions.

We'll begin by focusing on the second step.

Write a function that takes as input a **NumPy array of length 20** representing the **realized monthly demand** for each customer zone. Your function should use this demand to determine the **least-cost way to serve that demand**, using only the three active fulfillment centers (**Atlanta**, **Dallas**, and **Los Angeles**) and, if necessary, the third-party firm (at a cost of \$20 per unit).

In [32]:
def least_monthly_cost_fulfillment(realized_monthly_demand):
    active_centers = ['Atlanta', 'Dallas', 'Los Angeles']
    monthly_capacity = {center: capacity[center] / 12 for center in active_centers}
    remaining_capacity = monthly_capacity.copy()
    monthly_cost = 0
    TPF_per_unit_cost = 20 # per unit shipping cost if using third party firm

    zones = list(demand.keys())

    for i, zone in enumerate(zones):
        demand_units = realized_monthly_demand[i]
        shipping_costs_for_zone = {center: shipping_costs[(center, zone)] for center in active_centers}
        sorted_centers = sorted(shipping_costs_for_zone, key=shipping_costs_for_zone.get) # sorting centers with lowest cost first
        for center in sorted_centers:
            if demand_units <= 0:
                break
            if remaining_capacity[center] > 0:
                units_to_ship = min(demand_units, remaining_capacity[center])
                monthly_cost += shipping_costs_for_zone[center] * units_to_ship
                demand_units -= units_to_ship
                remaining_capacity[center] -= units_to_ship
            else: 
                monthly_cost += TPF_per_unit_cost * demand_units
                demand_units = 0
                
    return monthly_cost


#### Part b (20 points)

Use the function you wrote in Part a) to write a simulation that estimates the **expected monthly shipping cost** for SwiftShip.

- Run **100 replications**, where each replication simulates a single month of demand across all customer zones.
- Use a random seed of **8675309** for reproducibility.

At the end, report the following:
- An estimate of the **expected monthly shipping cost**
- A **95% confidence interval** for the expected monthly shipping cost
- An estimate of the **expected annual shipping cost** (based on your monthly estimate)


In [34]:
np.random.seed(8675309)

numReps = 100
monthly_costs = np.zeros(numReps)


for i in range(numReps):
    realized_monthly_demand = np.random.normal(
        loc=demand[z] / 12, scale=0.4 * demand[z] / 12, size=len(demand)
    )
    realized_monthly_demand = np.maximum(realized_monthly_demand, 0)
    
    monthly_costs[i] = least_monthly_cost_fulfillment(realized_monthly_demand)


print(f"Average Monthly Cost: {np.mean(monthly_costs)}")

#print a 95% confidence interval for the average monthly cost

avg_monthly_cost = np.mean(monthly_costs)
std_monthly_cost = np.std(monthly_costs)

conf_int_up = avg_monthly_cost + 1.96 * (std_monthly_cost / np.sqrt(numReps))
conf_int_down = avg_monthly_cost - 1.96 * (std_monthly_cost / np.sqrt(numReps))

print(f"95% Confidence Interval: ({conf_int_down}, {conf_int_up})")

# print the estimated expected annual shipping costs
annual_shipping_costs = np.mean(monthly_costs) * 12
print(f"Estimated Expected Annual Shipping Costs: {annual_shipping_costs}")




Average Monthly Cost: 92.38145889802873
95% Confidence Interval: (90.66550999053992, 94.09740780551753)
Estimated Expected Annual Shipping Costs: 1108.5775067763448


#### Part c) (5 points)
briefly compare your annual estimate from this simulation to the total shipping cost from your answer in Part 1. What do you notice?

#### Part d) (10 points)
Modify your previous simulation to calculate the probability that the monthly shipping costs exceeds \$140K.  How does this compare to what your answer in Q1 suggests?

#### Part e) (0 points) (Challenge only for fun.)
Can you think of a better optimization formulation for Q1 that would better capture some of the issues around uncertainty?  Write out the mathematical formulation formally below. 